In [1]:
import planeTracker as pt

In [2]:
planes = pt.get_planes()

In [4]:
pt.show_frame(planes['arrivals'])

,Source,Destination,alt,reg_number,flag
96,Stockholm Arlanda Airport,London Heathrow Airport,-76,EI-SIG,SE
26,Asturias Airport,London Gatwick Airport,0,EC-JZI,ES
62,Lanzarote Airport,London Gatwick Airport,0,G-UZHW,GB
50,Glasgow International Airport,London Heathrow Airport,0,G-EUYP,UK
63,Eleftherios Venizelos International Airport,London Heathrow Airport,0,G-NEOU,UK
84,Amsterdam Airport Schiphol,London Heathrow Airport,0,PH-BGM,NL
75,Henri Coanda International Airport,London Stansted Airport,0,G-RUKI,UK
91,Billund Airport,London Stansted Airport,274,EI-EBH,IE
81,Brussels Airport,London Heathrow Airport,350,OO-SNF,BE
33,Nice Cote d'Azur International Airport,London Heathrow Airport,762,G-EUPS,UK


In [5]:
pt.show_frame(planes['departures'])

,Source,Destination,alt,reg_number,flag
85,London Heathrow Airport,Amsterdam Airport Schiphol,-68,PH-EXG,NL
60,RAF Northolt,Dundee Airport,0,G-WZAP,UK
57,London Heathrow Airport,George Bush Intercontinental Airport,0,G-ZBKG,UK
38,London Heathrow Airport,Glasgow International Airport,0,G-EUXF,UK
99,London Heathrow Airport,Guangzhou Baiyun International Airport,0,B-1293,CN
58,London Heathrow Airport,Islamabad International Airport,0,G-VBEL,UK
14,London Gatwick Airport,Leonardo da Vinci-Fiumicino Airport,0,G-EZGH,GB
74,London Heathrow Airport,Logan International Airport,0,G-VLDY,UK
31,London Heathrow Airport,Newark Liberty International Airport,0,G-RAES,UK
39,London Gatwick Airport,Paris Charles de Gaulle Airport,0,G-EZBA,GB


In [6]:
pt.show_frame(planes['flyovers'])

,Source,Destination,alt,reg_number,flag
87,Amsterdam Airport Schiphol,Southampton Airport,2346,PH-EXL,NL
83,Amsterdam Airport Schiphol,Bristol Airport,6705,PH-EZA,NL
90,Edinburgh Airport,Brussels South Charleroi Airport,6941,EI-DPJ,IE
11,Amsterdam Airport Schiphol,Hartsfield-Jackson Atlanta International Airport,9753,PH-BKK,NL
9,Paris Charles de Gaulle Airport,Dublin Airport,10980,EI-DVK,IE
8,Vienna International Airport,Manchester Airport,11163,SP-RKI,IE
6,Nice Cote d'Azur International Airport,Dublin Airport,11582,EI-EVG,IE
5,Aeroport Geneve-Cointrin,Edinburgh Airport,11582,G-EZAG,UK
4,Frankfurt Airport,Manchester Airport,11582,D-AIBJ,DE
7,Paphos International Airport,Manchester Airport,11582,EI-EVN,IE


In [7]:
pt.show_frame(planes['mystery'])

,Source,Destination,alt,reg_number,flag
113,NaN,NaN,-45,G-KCIN,GB
71,NaN,NaN,-38,G-HITK,GB
47,NaN,NaN,0,G-OBOF,GB
72,NaN,NaN,0,G-WUKS,GB
76,NaN,NaN,0,FIRE1,NaN
77,NaN,NaN,0,LEADER5,NaN
80,NaN,NaN,0,M-USIC,IM
97,NaN,NaN,0,T7-SLXP,SM
112,NaN,NaN,0,N418WS,NaN
67,NaN,NaN,7,G-THDR,GB


In [8]:
pt.show_frame(planes['naughty'])

,Source,Destination,alt,reg_number,flag
